# Extract genres of the artists in the LFM 2b dataset

In [60]:
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy
import time
import sys
from ast import literal_eval
import signal

from datetime import datetime

import pickle

from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import os
from time import sleep  

In [61]:
SPOTIPY_CLIENT_ID=[]
SPOTIPY_CLIENT_SECRET=[]

n_spotify_clients=len(SPOTIPY_CLIENT_ID)
client_index=8
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID[client_index], 
                                                      client_secret=SPOTIPY_CLIENT_SECRET[client_index])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout= 5)

In [62]:
filenames = []
for (dirpath, dirnames, f) in os.walk(os.path.join('data', 'generated', 'LMF_2b')):
    filenames.extend(f)
    break

df_lst=[]
for f in filenames:
    df= pd.read_csv(os.path.join('data', 'generated', 'LMF_2b', f), index_col=0, converters={"artists": literal_eval})
    df_lst.append(df)
tracks_df= pd.concat(df_lst, axis=0) 
tracks_df= tracks_df.reset_index(drop=True)
tracks_df

,track_id,artists,release_date,release_date_precision
0,3l1hAufuGEuzTVyWpcw3wZ,"[(2d6NCBac0YDM7UthnFGPYz, Strumparnir)]",2000-01-01,day
1,7dDxqkwv7SeyYvNKHh3EP4,"[(1XVbGKrwpq5PkEbOX2z5mg, Lay Down Rotten)]",2007-12-11,day
2,1awRVWCeUiDvOJEeXRxcOv,"[(4zMX9gWf1DKhvsYHUCbBF7, T Bone Burnett)]",2006-05-11,day
3,3MezoVmZwrvnlWraxVcFTy,"[(2eJkpFtB62y7qCx27DDNGY, Dome)]",2019-05-31,day
4,74G15LOskPtxlTDQ2RepMI,"[(6XUNUZVWpFql1ysNCVvsNS, Daniel Tosh)]",2016-04-19,day
...,...,...,...,...
2159450,3gy99eFkZ1SnQCv66cxbxJ,"[(5gVgcz9X3QcrcrrDXR56NP, Psymun)]",2014-05-08,day
2159451,0wiLGzxHOjZEFTVocUOcYJ,"[(77C3owGTFPmigDDIpQLoiF, Rada)]",2007-01-01,day
2159452,0molN86dzbiya6JwHLSmhN,"[(1OpyZDaOsvR1SAYU4RXB5y, Psyche)]",2005-10-25,day
2159453,3eFmdipk73HxqRZYgCyDD3,"[(4ePunOqQbOYoQwd1298g3Z, Silver Jews)]",2008-06-17,day


## Get songs' genres

### Recover the songs whose artists have already been processed.

In [63]:
processed_songs= []
process_songs_lst_path= os.path.join('data','processed_songs_lmf_2b_V2.pkl')
if os.path.exists(process_songs_lst_path):
    with open(process_songs_lst_path, 'rb') as f:
        processed_songs = pickle.load(f)
len(processed_songs)

2159455

In [64]:
tracks_to_process= tracks_df[~tracks_df['track_id'].isin(processed_songs)]
tracks_to_process

,track_id,artists,release_date,release_date_precision


### Recover the artists that already have genres

#### Spotify chart artists

In [65]:
artist_df_path= os.path.join('data', 'artists_genres_2017_2022.csv')
artists_genres_df= None
processed_artists=[]
if os.path.exists(artist_df_path):
    artists_genres_df= pd.read_csv(artist_df_path, index_col=0)    
    processed_artists= list(artists_genres_df.index)

#### LMF 2b artists

In [66]:
filenames = []
for (dirpath, dirnames, f) in os.walk(os.path.join('data', 'generated', 'LMF_2b', 'artist_info_V2')):
    filenames.extend(f)
    break
filenames

if len(filenames)>0:
    df_lst=[]
    for f in filenames:
        df= pd.read_csv(os.path.join('data', 'generated', 'LMF_2b', 'artist_info_V2', f), index_col=0)
        df_lst.append(df)
    processed_lmf_2b_artists_df= pd.concat(df_lst, axis=0)    
    processed_lmf_2b_artists_df= processed_lmf_2b_artists_df.reset_index(drop=True)
    processed_artists= processed_artists + list(processed_lmf_2b_artists_df.index)

print(len(processed_artists))    

266479


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Pre-process genres

In [67]:
all_genres = sp.recommendation_genre_seeds()
all_genres= all_genres['genres']
all_genres += ['rap', 'pop rock', 'trap', 'ranchera', 'corrido', 'cumbia']
all_genres

['acoustic',
 'afrobeat',
 'alt-rock',
 'alternative',
 'ambient',
 'anime',
 'black-metal',
 'bluegrass',
 'blues',
 'bossanova',
 'brazil',
 'breakbeat',
 'british',
 'cantopop',
 'chicago-house',
 'children',
 'chill',
 'classical',
 'club',
 'comedy',
 'country',
 'dance',
 'dancehall',
 'death-metal',
 'deep-house',
 'detroit-techno',
 'disco',
 'disney',
 'drum-and-bass',
 'dub',
 'dubstep',
 'edm',
 'electro',
 'electronic',
 'emo',
 'folk',
 'forro',
 'french',
 'funk',
 'garage',
 'german',
 'gospel',
 'goth',
 'grindcore',
 'groove',
 'grunge',
 'guitar',
 'happy',
 'hard-rock',
 'hardcore',
 'hardstyle',
 'heavy-metal',
 'hip-hop',
 'holidays',
 'honky-tonk',
 'house',
 'idm',
 'indian',
 'indie',
 'indie-pop',
 'industrial',
 'iranian',
 'j-dance',
 'j-idol',
 'j-pop',
 'j-rock',
 'jazz',
 'k-pop',
 'kids',
 'latin',
 'latino',
 'malay',
 'mandopop',
 'metal',
 'metal-misc',
 'metalcore',
 'minimal-techno',
 'movies',
 'mpb',
 'new-age',
 'new-release',
 'opera',
 'pagode',

In [68]:
gender_converter = {'r&b':'r-n-b', 
                    'soundtrack':'soundtracks', 
                    'ost':'soundtracks', 
                    'urbano':'hip-hop', 
                    'britpop':'pop',
                   'orchestra':'soundtracks'}

In [69]:
print("Total songs to be crawled {}".format(len(tracks_to_process)))

Total songs to be crawled 0


In [70]:
# Register an handler for the timeout
def time_out_handler(signum, frame):
    print("Spotify call timeout!")
    raise Exception("end of time")
    
signal.signal(signal.SIGALRM, time_out_handler)

<function __main__.time_out_handler(signum, frame)>

In [71]:
tracks_to_process

,track_id,artists,release_date,release_date_precision


In [72]:
def save_artist_genres(artist_genres, processed_songs):
    artists_genres_df = pd.DataFrame.from_dict(artists_genres, orient='index')
    rename_cols= {}

    rename_cols[0]='artist_name'
    rename_cols[1]='popularity'
    rename_cols[2]='followers'
    
    for i in artists_genres_df.columns[3:]:
        rename_cols[i]= 'genre_{}'.format(i+1)

    
    artists_genres_df= artists_genres_df.rename(columns=rename_cols)

    now = datetime.now()
    dt_string = now.strftime("%Y%m%d_%H%M%S")
    artists_genres_df.to_csv(os.path.join('data', 'generated', 'LMF_2b', 'artist_info_V2', f'01c_LFM_2B_artist_info_{dt_string}.csv'))

    with open(process_songs_lst_path, 'wb') as f:
        pickle.dump(processed_songs, f)    

In [73]:
len(tracks_to_process)

0

In [49]:
artists_genres={}

query_size=50
artist_query=[]
for i, song_info in tqdm(tracks_to_process.iterrows()): 
    song_id= song_info['track_id']
    for artist_id, artist_name in song_info['artists']:   
        if (not artist_id in processed_artists) and (not artist_id in artist_query):
            artist_query.append(artist_id)
        
        if len(artist_query)>= query_size:
        
            signal.alarm(10) # Fire exeption after 10 seconds

            query_successfull= True
            try:
                artist_info_lst= sp.artists(artist_query)
            except:
                signal.alarm(0)
                query_successfull=False
                if len(artists_genres)>0:
                    print("Saving intermediate results...", end='')   
                    save_artist_genres(artists_genres, processed_songs)
                    print('DONE')
                    
                else:
                    print("No intermediate results to save")

                
                artists_genres={}      
                
                #Refresh client to use for the extraction
                client_index = (client_index + 1) % n_spotify_clients
                print(f"Using new client  #{client_index}/{n_spotify_clients}")
                client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID[client_index], 
                                                                      client_secret=SPOTIPY_CLIENT_SECRET[client_index])
                
                sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout= 5)
            
            signal.alarm(0)
            if query_successfull:
                artist_query= []
                try:
                    for artist_info in artist_info_lst['artists']:
                        artist_name= artist_info['name']
                        artist_code= artist_info['id']
                        artist_genres = ['hip-hop' if 'hip hop' in g else g for g in artist_info['genres']]
                        artist_genres += [g[2:] if g.startswith('j-')else g for g in artist_info['genres']]

                        artist_genres = [gender_converter.get(g,g) for g in artist_genres]
                        filtered_genres = [g for g in  artist_genres if g in all_genres]

                        split_genres= []
                        for g in artist_genres:
                            if len(g.split(' '))>1:
                                split_genres= split_genres + [g_aux for g_aux in g.split(' ')]

                        if len(split_genres)>0:
                            split_genres = [gender_converter.get(g,g) for g in split_genres]
                            filtered_split_genres = [g for g in split_genres if g in all_genres]

                            filtered_genres= filtered_genres + filtered_split_genres

                        if len(filtered_genres)==0:
                            artists_genres[artist_code]= [artist_name, artist_info['popularity'],artist_info['followers']['total']] + artist_info['genres']
                        else:
                            artists_genres[artist_code]= [artist_name, artist_info['popularity'],artist_info['followers']['total']] + [f for f in set(filtered_genres)]
                        
                        processed_artists.append(artist_code)

                except:
                    print(f"Error retrieving data from artist {artist_code}")
            

    processed_songs.append(song_info['track_id'])

0it [00:00, ?it/s]

In [50]:
save_artist_genres(artists_genres, processed_songs)

### Auxiliary code

In [92]:
target_songs= ['0LX2VNf5w4iOHW1yyIqb74', '6QNok4FrsyOuxFf2cPj1GB', '0tTS475qIqv3KXYZMXjsYy', '2bxxlINUlcMQQb39K7IopR', '6rYogEVj60BCIsLukpAnwr', '6xAuCvQ56HHJazssRDSLy3', '1ul8iLt2WnFe2UIyovjg7q', '04u3fc37nHFKN7GJTSIwI8', '3kgjDFMkBBMlr7DXoBYDA0', '4o7UpjgHjVnvHCbfH1sQVK', '2ZHJEckf6HJXMSuD2icSSW', '5XFyX7T46pLlApNj2duTdL', '5LeUKbWRby5hGRHktQkXQV', '4fEkbug6kZzzJ8eYX6Kbbp', '5TirRF3azWV5OpyufcDCFP', '5CkZIA3WpaEFxp0wSjMzRI', '4hkB2bR5ek6lJChj6aunCn', '7Fh934KVdTC7ekW6Rqangw', '2BVeB55HSYHIpcmNZkG8Fy', '61ZRdppZ5sCtl9m5gfhoGO', '1VrWc8r4AjrR8YfT0vlsc7', '0wmlHdnduAWTeHkzx4dX9O', '3kQ0LUzYBc3fzDVIpFhfuM', '5LUxu287CsPM7jy3yS3l86', '2fIUlieTjLTaNQUIKHX5B8', '4zEkMRxNAtvWhnaUr3ukOJ', '6xolQjWFT24ykWke55u9fU', '7H9jPV9qWyp6V629038aXU', '6w3SkAHYPsQ1bxV7VDlG5y', '3wTmekbMox022tiwirdy6F', '746Q2CaUN4Rfttp6D8rZc8', '5XJDexmWFLWOkjOEjOVX3e', '1cZQSpDsxgKIX2yW5OR9Ot', '52SusfPq6EW5PgrjUqMHSy', '2AbQwU2cuEGfD465wCXlg2', '5eZRINUbzwWkqy5UYhPVZd', '3egWSWp7Y4FyCKIyvXbw7L', '6zOyQHK37vQX62CFowxvWc', '28kGqx4hUF07ChEMz26up7', '1m2L1oj16MHAnZqzquzW4B', '6HqgxkTt9KGyt27zYrwVan', '4nXXIxTneJksvGXrlmX8oA', '04UpsyroM3wyQYwhYd9mSv', '7rTPCNINGzMp3Hc8Xqht5Z', '6G8I4rDBsRyYXqgvhvNZIt', '46wEUZyujVrFSrdCnTKQmV', '6zk8WqI12buD0B67tteQ5h', '0tC995Rfn9k2l7nqgCZsV7', '6OS2jyNVjuVsvA63jsw3Rf', '0kefKIzkoDXLQ4qmn72pu7', '3apNWJFN6ZANKwNvFK8An7', '2Ol461nHrkjAefbr0wL2Cm', '6OpOg5HVCc8xVf7OVrd5Fk', '57aMHpsX2SXTbLatmeEd6j', '0BLwluvlJt4aAE95gomnbe', '1ZhwEGHgNZCnEFPrlWFTMM', '5SWg48UK0FZTbr5Q3ubfD6', '4mN0qcMxWX8oToqfDPM5yV', '61XnjVwFLWusIb5USLymCm', '6dDaXbE1caD65TONToYXYx', '0Mum6waR8P9WICLVTpSi1d', '0ypwPd6ZLWkYuNVRnRFor2', '05CRzFTp7TouOXPuH6Tapu', '62lbdm09j0QUbWB7SqCJvt', '3KBONi330rJyUVTWrWI7xg', '57RYqNhEvaNiafuP1X73GS', '2boacFGRX1WIo7NFS8B0F2', '5DuxmiiW5nk7RERQqIkxey', '5RWzjEjUW2NWpSm00Ieo5X', '0g5iQz7W3rnBbGOdW8ZnNH', '1KnHjTHixdc2LUAao2idXo', '29nrwultxQ8jENVmXoyMqK', '347ZYa3M1wIhZ3j8Awnd19', '6q7ooPx2ZxV6MJ9I7d1zmn', '7uQVLqNyMFKlPMURUQXFXi', '6yI1aFdfAnjSHaDnHBFAD9', '1snhtMLeb2DYoMOcVbb8iB', '5uP9fBIVGSwIQJoGVDBRqk', '1znKr6bT7ANGqQOvTu8ubv', '6WQhHOv9wj8d0uSXzySUCU', '0o0PoOOyXcmqcyVz8aSdYu', '71Q2gPB29qmkDICBn5V9qz', '7rAjVJgwZhvmMIt5vfyNGn', '1QJbbsxg2wqidJj51d3otw', '3XxNRirzbjfLdDli06zMaB', '7IaezJ3t50nl6S1NISCgxj', '1ktRv5lOULehDAw5xhFIFz', '67pQ8Yr09zDDzzwWw3EG9R', '2POj5DK9ZiqcLkeTeTjH7b', '754BJcKdHZH7tj9xFq3hHO', '4uJNQGa3L2frXDxwgouTIw', '3zhijRRIZX2B6G2T7vJl9p', '190bkHbFrRvEhcB7Zpuv3y', '5Obu9ifz8EKdm0uL5CvgUY', '7uaikZmOtxXTNe5kPWv5t6', '06knYh538h5SI7OAEF8ek3', '6ifumpAV0R9kdjfy6pWzHc', '7waNCUQ1Ne7OoNHgqpgMZ7', '3zE5InDD74ETpPKj31203f', '7ljc6gTQ1x4Bx5GMWu93Ay', '1phfTBIocBW3UwqcYjaEN6', '4RXxwBtdt6k1YNyyAyiOng', '2AiZXMqpVlCDnz3trZbWaK', '2cusgVvXlY17oMoDAbcdqD', '0r63ReVRjxrS4ATbLrdcrL', '5p4B4WSKIGvSfhc90GMxw9', '2oQX8QiMXOyuqbcZEFsZfm', '1mmWhoTJeAARzU4ggjeOkX', '5K4W6rqBFWDnAN6FQUkS6x', '1Pwd1KP0QpMW7BZq6EdTB0', '46uaNKR4GSjmDEK732EeVw', '1AM52XHXnWAEqnQwpxTLWm', '08SuZpCtCHnOuW4ZzjEhsg', '0pOruKWwgyqSLZhn4Xovs9', '3FRVhQjAe73b3Et1XsOCnF', '7GtwhLS4bvGc7mkyWjPmpC', '7yeFMUrYTY5cAZx0GKXnti', '0lI3rF4hi4op6UxqlLHPzv', '6U2dgcU46Iy9pF3RQBj5hG', '1yWjbmwKNd1q0PI1V1BuQ7', '6AskOVL13sRjMWnL45lxp0', '6L3XsdwSNcPAPxHq9KuyxT', '0HW2OAV5XOPQgKoFx2Q6uJ', '4yZ8oMx1UD9PWn3M0SP8LS', '6VSZeMeJlVPGoR2nfB6UxD', '4d3kmfoZBTuUPSUjgP45uo', '1apmSb6aeHZyr9zUsy1Z9q', '0FDJB5xf8i09jDjIg1qNED', '4DrCrfFVwLbg8ZSvblp9Sf', '0zGt5wXkvlORfIk5tnFxAD', '2kN7bQzdCZkSBYhzF6k5lY', '1wFoE1RwBMWoWkXcFrCgsx', '1sVmXkzX2ukc6QvasrDBES', '1vqR17Iv8VFdzure1TAXEq', '3YrXRilGum4axHRdk7Grpt', '4Fn5clYETVpCxnIcFSZhr9', '4BNWanhw4AjSXjBm9L1Jzy', '4wT8HegeM2lfzDDquSXIRF', '6HwxMgE895sejjGFin9Gvm', '4rUYk0fV0Z4pOtwVbEAyK9', '13bDjug9N0pyv3ZUINjkDV', '3MjlXVCfmLdY9QQ2GCd7iA', '31H7jkHn502Qrh12FGTbld', '5SulO4l40qDuV9zUGLZx7n', '3tQ3S05OMjsRpkQr8CvIqU', '6AVfW2Y10P7zqLC6Z3O0xj', '2SzdXGFgRUgygnqrqmsW9W', '0GnuCXvnvVtX5DShryuyp9', '422Vo1v0NaW7ZVU8EwvhKW', '5M2zpiu3UmpkILAzG32bCP', '5fvTHKKzW44A9867nPDocM', '1GQ4ihUi4W36nt5ZKmtMgh', '7adgheUEf2iOUaJaz0o1YI', '6jwzGu5VdkR5tsRROYuxso', '658X86qMyJpSRY87oPvExU', '5dfZ5uSmzR7VQK0udbAVpf', '7CXlqxbfckEDH2hQMjrGyZ', '0Mdm2CCxStFoJr3GQGHf5V', '23ot0eI6ByBW6LrlBfr2bm', '2qMezKrAKRh2Z2uR3fCqHt', '5lEkQtWa3UKlI1hj7sktcd', '0ffKEtMDnROKWyJtXUnLbJ', '1S5J45qCGowVETVuq3vDyj', '19Y9kfsVkKLLIVRvhDMdTa', '1VJ0briNOlXRtJUAzoUJdt', '18hpynG63scJYxOSDYHUVN', '2MEWgRS9kjwxFXxbFBTnPO', '71yQbLQFugdyveqNvMrB4x', '7em66sKQ8BMUzCFr8zbSVP', '2XyVj5qsjy87RPXk1O5if3', '4EYftd8N7pujRjEkIxAZbE', '0DnjaQqb436AH1idffI6CQ', '2dQXoFzwfoxlh8xKIzegdI', '0fRvb5TzYdVo4qEUMWPaFu', '3gS0rQdigX9t4PJu1ay3BY', '1AgAVqo74e2q4FVvg0xpT7', '2oeTqp8DgAmtnP2WGc1OS2', '26EC7tAx17ojaMTykrS44u', '1bpca6RQE5kp92pIwPMBE8', '62zLrboqOTPyozwi721CVy', '215ciYy7TIYnGRim0I1nJ3', '6VNENoZhUeW7QMgHekaktJ', '2YzV3X4QE84VsCiIAHrC6D', '0NmtgATbZcejMnRU5ixeSs', '2qWK8K2Jfh67UqtwY8tCW6', '5hOWmWLLYJvAEuERZbyxpk', '1XCpuT9i3CT151EqxEshQK', '1Zb7mgt9TRL16FhZa6qhXf', '0Nt5ociYleLasTWpj28P6L', '3EuteIwWAP4mLq0kvLML99', '0vkpFzbSTmwPuGGnhsblFI', '2pvyE8W9RWESQxkyAWZqgY', '2eLrVxwm6t4QeaORq5w8VW', '3KNYn0ah22HXySPgPkMsrS', '4Fr8ee9ec47r9fC8SqiO8z', '1HzjYKBYzU5HamS57RbqhB', '2TVTUyOkmhEQpMd9BD27hk', '1EArbtCy5GrFydhn1Oq4hY', '63Ns0cyE751e5RlTCS2lpO', '13ySOy7rghkjVk5f5ChBW8', '7idPv1alc0mcEhEsKlMV2O', '6h5P3J0j1x2lhiDk9w4deZ', '2L3W8TmknCpxsOCiZQixyl', '01m2XZ7m7rAz6KY3scTdaV', '494DytwGz7wCII2IMjQGRa', '1BYjhAClGpBTLZfics0VRZ', '2pG0eQLB8CbmmWWmjJi4yd', '7hNo3SUYbUHGAdWmXyUPC6', '3tZdknQnzsVIIdaGCTj4RX', '6BeO1KuAWBqFriL1mLHtNc', '6mzs66iVW15C5iLt0JLt41', '7GkUskIBOKpJRyzpE7WQBD', '6bIKNXqAGE6GDjolShzq3l', '1mXAhKnZEdF6rotyyd4GBi', '1jsB2W4c9zTq9ilGXegrDM', '5Yuv1zhbDFuLlV3u4f7w28', '3VCrybIJKH7UurbDcZbMmn', '4w5vfLTFKwpd16ZzmYwcXA', '74OaRjmyh0XyRZsQQQ5l7c', '0FUsrstJwmg4WVHQMTYuUA', '0ljiBZ6hoKujOLCzMxm8ih', '3wwEkvc5v1R19fMX5fCj7a', '2HkyxGmwcNwdXWOawm0J8c', '4LyMWxYfZfeNcAl6vbEmQ3', '24jrxG0tKcwgAzsLuPzyMi', '3zh3U2eQ64EhBFbJuxgf1M', '1YrtUPrWcPfgdl9BaD9nhd', '4I1y8Rd0H2NGXMzmT6JWsz', '4f7KfxeHq9BiylGmyXepGt', '0eLoiWLwWeaHGNkYke5G1k', '2av4MrhPDUQLG3py1i0h7L', '2elBjNSdBE2Y3f0j1mjrql', '7KFOc3T4Xo8DVZt4PWw2qN', '6n4SsAp5VjvIBg3s9QCcPX', '1Mw40k757jZuiL0NIJpdO5', '2XjqKvB2Xz9IdyjWPIHaXi', '056KMTw6IztdQjBmFfVyO3', '2SabhGDiBSyaM6QSuBUVqB', '7wGgdCzwUJviKblg8VPlPL', '6pkv72pSnhPBKqqFzhDe6f', '1PaWbLip9XRxkmxmkzee1p', '1MdW7HZlXcgJKURqSvvcip', '7Fpb4yEAPfiQfY0Bx0uLm5', '0lIw3RIPiAC50dWNjJXe2Q', '0lFcTmk8mcrcjiZFzk2KgF', '5DdhWWd83hbrWLXIVx129X', '4YRxDV8wJFPHPTeXepOstw', '6YGhVeajpGcixgdaTfakUL', '3QLMLmISoZnFxUdzSoT6JB', '5k96hVqrpJoo0zmlVggVzv', '3CnCGFxXbOA8bAK54jR8js', '2ZMnWI3oIHyj7MyuZ99gvu', '7iK8PXO48WeuP03g8YR51W', '2PC0CLpUsoEQPNIZKg2ZX0', '2Cusk8DpjH8r3BDvuN3ufo', '7ANf7tAnPESElMa8bUTMmd', '6vDgr8a0h4QTSFQTZy1dua', '47uyFQHOD02S0lj9ptRpoB', '4qla74GaFZkhFIUQvGZAEN', '0aDZC2CPSqgpZwzqrh0DeF', '0dz4rGLXin0gvKYqFh7iyI', '4y9i0THzpXUR3gdaeHNME1', '0BveztSEpa980ImsnzgdmN', '0sSxphmGskGCKlwB9xa6WU', '71thoyIIWlcK2fL2dCSsCE', '5G6LaA5I0P51M49dzJV5k3', '2jTTRLGcvNfqOSxVw4Ipxe', '4SXx5pMoj7uGi1WPcZhvXA', '6J434ZmyHxFcBjHXAZL9QY', '5j9mMB9gew9wNCvFJWiNho', '7ajcXCqrSRoMtviBDVIjqn', '2kkQthS9OLpK4UqNWYqoVl', '3OLGltG8UPIea8sA4w0yg0', '5y51EHpzGr7CwrhAnO74Gp', '2MMPzBpMeCZVPRnV5mddc3', '6qMCbcnBinopCiiCrKQlF3', '36SvDHIiV7VNyLLmChI1H6', '438nTuoZFCZCyBl33jE9dU', '7kFoHyLeI8IWSoKZ8uIeY1', '3xYFPpQA4Hp8BTusV6um2e', '0nvdwVbj7NT1WL9P8JowLD', '27Vx7FzWdRbnFkCKI4QlWV', '0XuMowqsbFr0TmhOHfmWuc', '2jqTyPt0UZGrthPF4KMpeN', '6fm51xYlbaaMz9YQhTLkFu', '7pSH4sO2lXAxFKF6MkwORv', '71k6SmcYKiIPq8M2RxK2Jr', '4GAS1OgXh8D5pEA6s6eBNW', '1dvuibBCx9TnbCKIdOEF4l', '63YmDLaZHbrYEf26oOS2JS', '3ZZlaq6tv1IcMjNtrZpsLd', '5C4PDR4LnhZTbVnKWXuDKD', '48RCl7AbbfhC60EN8a80Op', '2510ceejHZpIpxbGpiovch', '6zerBCeTLTvR5tH4oEBRRw', '24Lqq3mnesl7IljSVFEBEO', '762310PdDnwsDxAQxzQkfX', '1mBydYMVBECdDmMfE2sEUO', '3vHwJg9XHrACvX9ZlwV8pT', '4FmQBODJcAUMIcdSLdXf3o', '5RmQ8k4l3HZ8JoPb4mNsML', '3b5bg1k6N9u31OtzSfK2dP', '0PhqM7UAxtvWYi5j4MwxSl', '3qeuv2ijwaAcbX3KVMJfuj', '5VLk4bLGTtJkVCGPNIaPqQ', '78IEFKJN8gQgJM1WOvt2Tl', '0UTsdD7VIsEB9i7a6DDx5t', '6hkKbkZGvAXuvle2FhCnxy', '27RxkAdGThrZ2VisW6a1mN', '7fvEDuYk5vkKKZGpv6Knus', '7xOuCEpAtEfskZc1wnxlsD', '4hw9WaVkOzRQN6XaUqULv1', '6NRt60NeMuVKeqzUanbHfx', '4NivXml482bxwjzaqeXGTi', '7b79bQFziJFedJb75k6hFt', '7FvMAbdiffitFmbuJN3Vsk', '7yK3vix9XmeNwPDmjGs78F', '7MBNcpWHn8RTZFUQv5fNPp', '6fCylnBqt47fMNEPaANT2B', '0t9IVyFGzH79wIFfhEXddQ', '6HQjCmn6AnJdHXSac0xPwj', '4nXezpatStChCEWVvYLjd5', '0rQ69yrbz7CeUmXUn1beIj', '6XjHGcba7ZbaZ6nYtwhaCj', '2GFsWXpTIn6G1kRjW3RAQX', '59pq5Vw16UDkR9txoPKv86', '4G8DaRnN7oPFkEJXwX2i94', '7hIqJfRYGBWWT1Qxu6Cpd2', '1pQWsZQehhS4wavwh7Fnxd', '7qUmZzfGAo66NPgOEx0MWa', '39KEQQAG9JhIt71g9NEVhh', '1tqysapcCh1lWEAc9dIFpa', '6yDYti6aQsOfmLqBZ3Tu25', '5iZ6jMDkRa7RKLQplJuQUC', '4qFxP3qN9GsnZDOkAE6x2m', '09CY8fzqhZHR7rQAULoreI', '3d1I45VCwwwQ6Qi53vKAHt', '27Yc5RzJf27tJfqezJnHY1', '0Cm90jv892OeEegB3ELmvN', '0y59o4v8uw5crbN9M3JiL1', '3uFX3kMxtOUFMDaJoFPJwo', '2DspEsT7UXGKd2VaaedgG4', '4LLrhUqUv2Mi1NhHz0pjXd', '7DgcAqP1VQJvOJcXc7yPsx', '6fMZytDgX1Q9OV6ndSugym', '3xoBujDX2nT7UkO85fmzSb', '4Ai0pGz6GhQavjzaRhPTvz', '3wrzELDiDtiHAlLlAG423O', '7iXYHQPYLQSz3b3hbPJpk3', '13ZSDoW7YXmwyDpPDgtuCi', '2EsTolBYIEICM1kx7MGluy', '7hxYyECW5s96Y5qgZm135o', '3PtEOX0PJSh7ndOL4tP0NR', '0QHGCPmM4UgeNvrNPntSlu', '3oURu2k7q6hmeftezvW6xJ', '1G4EpdOVG07CIlZY8hz59F', '2v5eqze59fqRes2e5QVGhf', '2oM7LqPaoCUcHHHnsTtgn1', '7CCjtD0hCK005Bvg2WG1a7', '70paW48PtCtUjtndElrjrL', '3IwaLzOzILWyMzjXFWiDbW', '0FQt8Hdz5S1JE7By1rmVwb', '3IW7ScrzXmPvZhB27hmfgy', '5DAMo6bEAd0PFnP9TRNea4', '4MUb8ilmrxyePXwSkG31lC', '1IDuSpntFl2Mutofvrrutc', '2DePhg6Xy6JqwqPmFkvWtN', '6CstMzueHNF7UABHsyx7gK', '41Vy9bg5eOy5RWPiTE8pBI', '4uGlaZn0LSybHkHFEKBOV2', '7Icsejk4pdIhkq2KO5A0jD', '02Tq76MwpeoRu3BHIAiaio', '6UCQYrcJ6wab6gnQ89OJFh', '68gYAqni9tSrACmLCp4qoM', '64J3ISAKbfg1CuP4CVCMlF', '7AfDbA1rEHwRhzPH6jHSUE', '0V2DfUrZvBuUReS1LFo5ZI', '5kwthnxNdfnqGk0nL35wDC', '7L0jOBGO246iJcBr8uWfXs', '1gm1katIowFM22Ldqcw6DK', '5UopXhshFFqRIMfeZrBclq', '3TUlXyaZ6a8oyjJY9T5Zvf', '3tuushgEVJSXAkxR4ZxOhJ', '6XmbeFgAh0itkfnnLXza1J', '4CHiVarfTsFhkFOk5vHS77', '6tYuW96NhiYI3sAxEDsqK4', '3nuhBdXXO0mnjugdrDxkmt', '029rFJVCRchYobJ2WlKWuE', '0yHhVZZRkJNbKmWGq8nciC', '6sdq9VmQyqzh3cE6msHaXP', '4FuonFF3VstySELPvJpBk6', '5OWPM4ybplRvlK9RJgAtSm', '03Yb3iBy9GCifXiATEFcit', '2qmA5QmaGZH3ky4qq25d1m', '0pnd3MP2rxAzljR9AqXUJB', '0EPzUAW8kwuPedmmVP6n9S', '4ynaUZA4BVF6d2XaNuflVz', '2owjgBZQ70LoCNHIgW3gSk', '5KBbs98MK59UNMF1VTwMmz', '7Dx7RhX0mFuXhCOUgB01uM', '0oslLqFFiQGt023MVrPyhd', '1FVZNAWNizA5FuYp5UvYOL', '7pbijUBmXdie5ChABSM4F7', '157ntmMB3KJcSN5GetlzJk', '1NBkKa2SgVP0uirl5vAB8t', '6Tz0QRoe083BcOo2YbG9lV', '5VwMP2dPEhREdi52qKyLTn', '4oopAFtLdKXCoq2XoEjDZD', '0o75YCEpFIA5hy8g4DJCYR', '4C29ETLBPmYKYwtJUepbJz', '3yMtvgD2LCo6Ws4Z08fTFj', '3a5ydeAaojKa3CHWe5PVWK', '6kq4GHwUcUojGIu0ziSNXf', '2F5W6Rsxwzg0plQ0w8dSyt', '4BFLElxtBEdsdwGA1kHTsx', '67t8LmqkkLHvzGjAlEdHBO', '3gkbRDj7pkIylWVBpRfrQf', '5NlrJ3pca2bOCx0PTfVqH2', '1aNVLXFIfADxi6ZUcsdVH3', '3rQXUpppnyxRkQNxoLFtUV', '3I6Jz2oJE6aLXSYNn6SDZR', '2D8swU349pR0GOIpwBg4hy', '1LEtM3AleYg1xabW6CRkpi', '5f6nz3iqzrfiUfKOIKvLvd', '6NAhbfEqeWELwfjH391Vz0', '3lkxYq1XVRymJJNgHuzA5R', '1iFBAUJba6rHRgFNOJYUDD', '4iCzh7b2cLbHVsPOwhr8W0', '46XcyK8FnyCJJlvYCUwVZH', '1ErDfqRJ7wjrkm7Qpsnrmz', '07YUOmWljBTXwIseAUd9TW', '0Jsk5iYMr5aNjHury7blm1', '3qIZg3GhHlbuCkPYwa7h95', '7FXHLGDZv4fx2vg3LHLoGJ', '4zaslnpNr10iUFJGx0XqpM', '6PD6eSZM8ulCg5PRU6mEII', '35HTycyWifnoCzK2EQ3KUX', '6CawoDDP1IZUSGl4wSJGC9', '2cODK5X9w3qXuhLQ97LLyi', '0oO1IaDOBSeI96HbnCa5pZ', '2JJeLxFOub3q1h6GTKOOBi', '7JMBJmGMqw4H33HECyW4QP', '00RJAkLnjGx4kVWVJbOJx1', '7a6TPrZH6MH6NFR0mlZLOg', '2UP00KR2x3TuAoPkZV1O6Q', '7FqkRutc4zWMrnEAUv3Xwd', '6VLldomvrVfwjvfLdFDq0o', '0Ntl0oWMPWfBOoi9Qcr9ht', '0XNvr6bnNywV946ZKaJtr0', '0tmwSHipWxN12fsoLcFU3B', '5344K3N7rx7kw1HjO8psuq', '7kCgo900pQKCYI7eNw6yJ8', '5jfz7uWPwf03hdEewW8AI8', '0cA6pnat68Y6cXfNZMtzDZ', '3ZwM56IXFGMIsUFK0GEygo', '1vKXVGgIDP1XXZX4Tehjtn', '0hk4xVujcyOr6USD95wcWb', '18CJ8k3h2Rggioow01dlwP', '73nFLjE3BcdgywGYVtvXdL', '1aeGU82zQacdkJrztnB7yF', '143I1xeiJLsFTxHvcDHSEr', '2dAKW1tsAariQ0Ta2gqp8N', '7J7IAR4rkzo4FcZTDid3Jk', '1pkDzgHvVqC17qYGKfLPpb', '0KPX4Ucy9dk82uj4GpKesn', '7kIg00UV1mRLC4BgfGlCwH', '4reUTWYBJ5qo98SW1sXMGv', '062AuVsQPIt8pXzbENjI9p', '2rYxEnHjY4kAGMy2muATL0', '1EWnP7piatUeXggTXofm5g', '6cC3KmZOI4EG7a94PtKEYb', '6f5kUMXGROFtdAtxXZwing', '0UhUyqIGtwYPQCCecADVqq', '1sICjVlCMOCah0snHXWUz0', '0EmioUrT5ZIveHOQnDcTGM', '5SwsSD9zyWaWeVyK9qIW9l', '7xNYY1Zkb1vks5m9ATlJok', '7Lmrb6KcIzfkmgbtokjsAL', '6xg3PSVEwviVyfnx3mtsPL', '5Ey79g5HdXRYhqZVCz6UD1', '3OknZgTUEWAu64xUAxUW7C', '5fIKQDlnegXI1qxcG3Kv9o', '1YwMnnHvnG64fJxYSVwmN6', '0BwMUrS55qLWdEyGbmBgkW', '3i6za4KlTDNe8mducqnwJi', '0Lvb0REREEhoj4mFcnkcI2', '6ybOB74nYTUpM6VGiP1egr', '0Ye4nfYAA91T1X56gnlXAA', '2kRfqPViCqYdSGhYSM9R0Q', '2yRhS7HafDp8QUAjiIsofx', '6QfFTZJHFSe9Xyes6DkAli', '4vnAcwsYeJD9QcQsowEuKN', '6rYEqmajzlhGVaayOJ2bpJ', '5W9RX9DoA3MSvQGnoIgs7X', '51o6QF11Nswx5jMlp6rAL8', '7fT2Me47PQ8T7954PKrcwR', '1lel1kMN3SUvXOtrO2tmfr', '5E62kscxqQ6HRcRRsxm1n1', '3oOJYwLB8FGSKYOQ5zx7Kk', '4okcJqKIn8TGpUDaDLDlRl', '1TDrOKmpx1jHdOhJHbngJz', '6MBiPjSFsYijvVDtKDwVs4', '50DjkxUjr341CAWEf6mEXS', '2Jb9jVnCpWkXtoGznFJ6bF', '1Xylc3o4UrD53lo9CvFvVg', '1jSjfSgDjedJdi5MoyRu78', '16wuk79V8hVmB6zUmcmUfF', '5rBUMBV7L6zPsEKnlgiOm9', '7Ew1APZDJuOdRQSPRvYP9x', '233YXmma3OPL1trG1x3WpS', '5jTInrFPBccvStXE3PP5l5', '5O46r5EwzdUL8RR1aM6yU2', '5Y6i2BGTpT4utAQF3L72rS', '1cYhx7ZOhYoVmnDPb9KMwo', '4zvLVP1siaenGVnUBDo2nt', '1U449OOb70EZlElNjLMwCM', '5Ptz1xkszwS7JBksmx8gMt', '4c5YsU2iQX3LvAKdPa0A8P', '3vlAUneucinIokaOUeYbrv', '2iFKcghd6GTtAKEe4eWPP6', '3omyT5zHz9QmE63xEk10OV', '1LAHwG1lENAABT6oT8yYXh', '03Qx159jmT9mvmw5YjzvAs', '0p0V7LW8i3S22J7xErqdmt', '0Odg6UKDVyJQruefzF19FU', '3WJZNXeqoLWzGRp7DvnnpD', '3ucYybVr1mTuB04HtoBDCc', '3WreFZLJqFGK3VJbxYewaU', '3GNq6JwYWh8B2xujy6ub04', '6NEj7qdWpQXSeiST36X7ES', '5Tx33KxzeDUCiJmJaaiVUY', '6mrK5l7yeX4E2h6SaTqIrN', '5f5z4wggh1hbr6Dg0BnoCe', '5eF9oy3uIgTMRIX3xkAfGi', '39HJXjH5hKcCzaU0g6mv8G', '2otUckChxWf2cp6UP2cD6e', '3NP4jJcW3R6qO6rbtnH0wn', '5kLXha9ScmLBu4DP2d3RXK', '3IYNUniWKkMmHKUHssC392', '6sge2FzdYxWSjWc5sBjmoy', '0fM4WhKpjJZc3a0eX947Tc', '2Osj5g9VkHReOlZgfoEQao', '0W8WpLB5WoXLgiA193LXk6', '7JHeLAp4gfLDbHnvh2cCap', '2WhGojaMjv4VVn2HQSqrLM', '3IUAZiICL3J7GlHYPgT414', '3vQ0GE3mI0dAaxIMYe5g7z', '6URykZzv72st8HDMcJNrsV', '32gBQPn4CkmUE9NzK0IXdX', '0lLY20XpZ9yDobkbHI7u1y', '6INLZbPHXGj6ERrjFGPYD6', '3Xvu1Rur46ZO8HiE9hBuxl', '0kh9Gvy9lGZsq84x7I37DC', '0m3D4grap8VFSzbJMqgNVk', '0hHSuSCpKFLU5twpJNTO3t', '26XxGE8hLV1t9IYa8872lQ', '7mhiQoeJ3jIil3BONfEtTb', '18k5EBIjwzfmthbQymZBhU', '50MDyQulQMbVbC4wf8T9kz', '5YC0dzAs461dkiWVPVqTdU', '4RsRmCGtK9yJmtQulq4lBo', '2AHugDX8XYbm2SNxbvptQW', '4iFznCrxI2nlVer46cgvWG', '5NmRijhUHZnaADekOLcOyl', '7BWNRZyZ9mhgp5t0m7Ny0n', '0gMAKyA5gWUJiVU5S082KT', '0OQMxt6l70FM2YlLwYDiEn', '4bWGzsNAtfPp8P7pSBukH8', '0WDJa0qnagyOnMaiD26wht', '7MVZzlVkfT4hgk6huk8IA6', '5VLVogbPeBgFHXQPZQzBej', '0sFpREh3MXhfRxOFZVStVG', '1INuLZXjjVbcJRyWvD1iSq', '23TFHmajVfBtlRx5MXqgoz', '17jayca0vxI4ZMPhGjgt4J', '6ltVunYjAAD70YtVO6rxvX', '0QZTQYBwBC1A951YedFq2c', '4zvO09rVUIVTeALhs6xLoB', '6fs2or0cKLEM2xohWq8SoX', '0QR764k0D36npmTMWx5bft', '4nWTnnYvLRayR3nwU2xbNx', '1HuverWDqGbFwmr8rBzPdC', '45Hr5etuwmSx4gSyn5qSeW', '5TmW0cFVEYEbypgTdSNNDn', '4VgfrD5wuAoN428fBZNSyW', '5yES5e4EJpUB0tfmWi3eao', '5gGmKXhM6UfXzLY7momhGY', '38t0Qk7AJg7YdrXmOC6TH1', '0Yb5TIkoemd0B5QzuoVndb', '4lAYuj2FX6djIqkxuDNvr1', '4cgBCGxtlfap2g6jveB7du', '790FomKkXshlbRYZFtlgla', '6DTzn5I7ckMyeB2ChPQCef', '3JXHU7lWqihROwbEZ04Xdq', '4VuARVJiWGsH2msKTdsKdQ', '6ytOHdKh4xt4YvF7tz8Zcv', '466bAN87QCwMSTBCCRzZ1v', '3Eb6uRsrydygbIIU25vCaV', '5FxZkV0729LOnA5LlwoC8g', '1dBKe7HabNnrGm5qJjHEzO', '0WJbT6T0nkWfYO5zEbUPOG', '2XMykstxpJ2hMSOj4IRpqi', '6fsAIzQPt8QSvdsrOtMKBS', '5uWSTChLUg0pNhLuAA13H3', '5KPJMJR9PCfMWSfco8i4W4', '7DBIKXxSh2BMPqSN7UB4rS', '0us4jbsSY1fxYXRUyEJW4K', '59fYphinaqBfGBtrQ4p9Hv', '3j4yuzqVqHMg99jtnQnKGn', '2gAcO16AKQdceAd36Oxe18', '4m6ubhNsdwF4psNf3R8kwR', '6TTVdg5UOUHsyiYPHgxLaq', '6pwVHVJPBAWa6bczsHapxv', '1HHcnYrMqv4xpwHJAzUfqO', '2NDJ7HCIU7mMP3odXsibY4', '2rd4gPVTRgvoI6H4xZiNZH', '4rbj52UIyw3wd42EMaS0Tp', '1kLqxXjyc7OwDJopftAH86', '6IuS73qKuYfrYqA5g5iq0h', '6acByqWJeuaM0TX5VoY6an', '7BgEOZ9w3Y4IMShXTMu1nN', '4bw2Am3p9ji3mYsXNXtQcd', '6VVSjZwPAHNbkKeMhTI2bb', '2iK1rsbYstkSVn57M4s8ut', '7J2gs5q2wLq6lU4q4wkyuV', '57VQcinZwig4ds83bgKxVI', '0tqxh7MvZ301BVW4e0p3Sa', '0bPOOMpOPbgBrsOrt1Sfvi', '4EcIU574ksr6mC1GMEOe0p', '7iO6zNSjgC7jpRivXZGwdI', '1T0F05F7TEo6QYr4vtGJdb', '2QU0UNvQvEZ2fuRUhkUNq4', '3WnmyxLdTHKN83h75tcb8P', '5pughe5rcsOq3GF0utMOs5', '6uQiTAaIF0UBTED52UMcZp', '1Kjs5u8GQf6zCFdTj6SI9E', '0MbvVdTPf7DeKc1DIC9Sav', '0355QJAVtxYiHEI9HJtoOO', '3OFPaIvw32yimTGrgVThuJ', '1jOps4FF4EvwPvtqWXjdRI', '03WNJB1zKyqW8DimkHP4Bg', '3D54NYRxCO8nmKhYQZFpPs', '7wUGouw4DxVZkM6dXAnuiH', '3XC7vn6odSKiwiRDloI9Vs', '6aGmKxXoKrSdovRUn8MBhZ', '3L1jDbMK9J4Rd6XVv0KDjS', '1umTeYMbiCSj1nl2fqXVcZ', '51o7b8jVB4q5U9qy41aYwJ', '0BeixPyuDxNBW6ZOo7YimX', '79DSCFvlXAsjytJCUMZ4tb', '5Cg1uhz6CxzXKvguhnoFAp', '1CQUPR60ihhBANxqwTZv8O', '3ptY5WTxu77JpAQQnHyxEl', '1dbp04JzZ81JXlMGupCnJ8', '1exwNqOyM56Tteu53lnUzW', '5xIOUIBQhGFX7HIj8lhdyU', '4RAbHb0oHO62If4S7h18L0', '1QsdzIKkTT5gDFj8GB1cIX', '2MsSx2rD8ZrRQ5ShTnLE22', '1hwKCeSXpLrh4OQND6iQHw', '6yhD1KjhLxIETFF7vIRf8B', '77ziqFxp5gaInVrF2lj4ht', '3wtMPMvPtiFylbnNXF6CAj', '5OZXWMwDhlYBRvoOfcX0sk', '5XyX171jkcvGEx4jkcZeLN', '3EsjO0y0DE1GC453Sgyr7Z', '6wBoKKHhGDrxVtp6XMFpIP', '7tkMUE1aCZQssaTYKdgmry', '39wUlmskWbXVwU5Ttw2cJL', '2KkO9uXHF9BVNJASjLekAc', '3G3tWOXo2TKCDJvrD48B6p', '47RMRcPX8zWGaiMSZRuOd4', '1UCxzrM5WJh9ure5vCcsim', '16vfg5F2MKku9FMIrFIqry', '2ZXnTEyYopSLCDiz5Z0XIf', '45XBPUL8jsl9sXvf44F9Ja', '05brE7yGJDmIYlMirfSi0B', '19i93sA0D7yS9dYoVNBqAA', '3qYiG2OKLj2WAKQVdIbigB', '2hnyMG1DuXarlHFJDITc3n', '2iB8eFB6flwQculkUrRssi', '30qOUeQ7zwlxGl3Fl9LNzA', '6PcYcDQ7UCL4U5PopumiWY', '1YmVpQw07GNZBW1M6tbNHt', '2YtAW3MRFFdRpHOyJjd3bT', '3abT87tqQ4Q5PA5nw6CYyH', '6fYEeLBc96CguiLq9FPBQU', '1DVPC5U9BGzbKCUVjvVzBy', '7CkfnjbIBJisquAJCZv6oU', '1XKjtpH5P81gpOXDB91IEB', '7cm5iwu932G8LwzcTM0afg', '7699UdGcD8ESAKxTcrwKvj', '1dlQ9EIUyQvXZ3l05YMQdT', '0zZJhc1T0zBurhnBwQ2fcu', '2KxjMGXFgX1vt9grpbD3xI', '2QEjAULvTmuy5eQBHcIpzf', '1ju3JsvJuNAMjxoIqaSP7R', '0vTUa2hbNHke3omoGxuKFm', '3hyEbRtp617pNCuuQjyOmc', '4heo640Ck89UJDhedui2IT', '1uiEaU2CF0ypsPwXXYrm7V', '16plk1BhihSieDutGFbLt8', '4puAp107dCehraE47QXVQX', '1XShZr5TXgAgMWnpp4mf33', '3bAFvRjPZrpRz9Ox1sElQa', '68X4w2Foj2kgJ6niUML4nA', '4sbXXFzEWJY2zsZjelerjX', '7crp1tZcefnjT5RuL6WZQ0', '1Ts9of7VPZElwPQnqnDSfW', '17aoRCzkZM3IFEbFqVCuX5', '6289Bbkkk3gaCbh1K7Rv8F', '20j1uz7iDOtIvIzCr5S8nx', '2PkjYgKhHI4qofYoxWB9ZP', '2IDLDx25HU1nQMKde4n61a', '4ec09tknYr0531ivdBMNIg', '3dCwcQ0B6yGThCdTSfSho6', '7zPS3i8YJBNeDcqXUHfCMr', '4eYxL9XWc2ZazbttqEvxJo', '6ieK9Ddas29reReO60on3o', '5VKmpjKvs9TorQyO02WvlW', '65ddsrNChT3wol9kEIJqKY', '1kfWoWgCugPkyxQP8lkRlY', '2UzjSnmjccLgd1DkvAH5OR', '2wkoKEfS6dXwThbyTnZWFU', '4wP1kxjUsc9IR4Iy2smL7o', '2kxP07DLgs4xlWz8YHlvfh', '64g25AtXULqxSKVwPXI92m', '2LUi4eKMEFKFM4awgurcSs', '54QmrKYac7fH0EHiTX8oyg', '0U1vPNbcJ2OJIcqFmhf3dy', '2ap4aQTG8zcKmzRjqlVmMd', '2u8P7EawurNYoIzRtr5Knk', '5ebPSE9YI5aLeZ1Z2gkqjn', '4UDaVZj9eMOfZFEGrcrwqv', '6P5Tw2kwv3BAa3iONsQe9x', '4IEJn7FhTQoNgvSlBH8H6n', '6Z3GmozKVgFxFcP2Qzp0Fp', '7x2hBF3bNPwW9PnKxg9Zf8', '7qM9nMv6HFHFUpiN034R3z', '2ZrN4KRe7Qw6boB8HN8L92', '6JFRI91YaCXREGQYzHSnUH', '0OYspJRShx3Yx7QIykOJgq', '1bAftSH8umNcGZ0uyV7LMg', '6qAoh45c4dsSqxNhp845SQ', '3Nbij6lwrwz0XB7HubxdN2', '7aRC4L63dBn3CiLDuWaLSI']


In [93]:
len(target_songs)

746

In [94]:
n_batches= int(len(target_songs)/50)

In [95]:
artists_genres={}

query_size=50
for i  in tqdm(range(0, n_batches)):
    artist_query= target_songs[i*50:(i+1)*50]
         
    signal.alarm(10) # Fire exeption after 10 seconds

    query_successfull= True
    try:
        artist_info_lst= sp.artists(artist_query)
    except:
        signal.alarm(0)
        query_successfull=False
        if len(artists_genres)>0:
            print("Saving intermediate results...", end='')   
            save_artist_genres(artists_genres, processed_songs)
            print('DONE')
                    
        else:
            print("No intermediate results to save")

                
        artists_genres={}      
                
        #Refresh client to use for the extraction
        client_index = (client_index + 1) % n_spotify_clients
        print(f"Using new client  #{client_index}/{n_spotify_clients}")
        client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID[client_index], 
                                                                      client_secret=SPOTIPY_CLIENT_SECRET[client_index])
                
        sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout= 5)
            
    signal.alarm(0)
    if query_successfull:
        artist_query= []
        try:
            for artist_info in artist_info_lst['artists']:
                artist_name= artist_info['name']
                artist_code= artist_info['id']
                artist_genres = ['hip-hop' if 'hip hop' in g else g for g in artist_info['genres']]
                artist_genres += [g[2:] if g.startswith('j-')else g for g in artist_info['genres']]

                artist_genres = [gender_converter.get(g,g) for g in artist_genres]
                filtered_genres = [g for g in  artist_genres if g in all_genres]

                split_genres= []
                for g in artist_genres:
                    if len(g.split(' '))>1:
                        split_genres= split_genres + [g_aux for g_aux in g.split(' ')]

                if len(split_genres)>0:
                    split_genres = [gender_converter.get(g,g) for g in split_genres]
                    filtered_split_genres = [g for g in split_genres if g in all_genres]

                    filtered_genres= filtered_genres + filtered_split_genres

                if len(filtered_genres)==0:
                    artists_genres[artist_code]= [artist_name, artist_info['popularity'],artist_info['followers']['total']] + artist_info['genres']
                else:
                    artists_genres[artist_code]= [artist_name, artist_info['popularity'],artist_info['followers']['total']] + [f for f in set(filtered_genres)]
                        
                processed_artists.append(artist_code)

        except:
            print(f"Error retrieving data from artist {artist_code}")
            
processed_songs.append(song_info['track_id'])

  0%|          | 0/14 [00:00<?, ?it/s]

In [96]:
save_artist_genres(artists_genres, processed_songs)

In [97]:
print("That's all folks!")

That's all folks!
